<img src="https://universidaddelaciudad.bue.edu.ar/pluginfile.php/1/theme_universidadelaciudad/logo/1756916448/Udelaciudad-logo-preferencial_de%20Bs%20As.png" alt="Logo Udelaciudad" width="200">
<h1>Programación Avanzada en Ciencia de Datos</h1>
<h2>TRABAJO PRÁCTICO Segundo Módulo</h2>
<h2>Pablo Ciociano</h2>

Importación de librerías

In [25]:
import os
import requests
import sqlite3
import pandas as pd
import plotly.express as px
from IPython.display import HTML


Descargo el archivo y lo guardo en data/facturacion.csv

In [26]:
# Crear carpeta data si no existe
os.makedirs("data", exist_ok=True)

url = "http://datos.salud.gob.ar/dataset/9d6315aa-6ba3-4ac2-acfd-f26233cba241/resource/148185dd-d9ae-4b0c-ae79-4f8e7f0fc2d5/download/facturacion-de-hospitales-publicos-de-gestion-descentralizada-20181217.csv"
file_path = "data/facturacion.csv"

r = requests.get(url)
with open(file_path, "wb") as f:
    f.write(r.content)


Limpieza y preparación de los datos

In [3]:
df = pd.read_csv("data/facturacion.csv")

# Convertir fecha a datetime, filas inválidas se vuelven NaT
df['fecha_deposito'] = pd.to_datetime(df['fecha_deposito'], errors='coerce')

# Mantener solo filas con datos válidos
df = df[
    df['fecha_deposito'].notna() &
    df['codigo_refes'].notna() &
    df['id_provincia'].notna() &
    df['provincia'].notna()
]

# Crear columna periodo AAAAMM
df['periodo'] = df['fecha_deposito'].dt.strftime('%Y%m')

df.head()

,fecha_deposito,numero_expediente,importe_aprobado,nombre_hospital,codigo_refes,id_provincia,provincia,periodo
0,2018-01-05,10972/2016,832.0,HOSPITAL ZONAL GENERAL DE AGUDOS SAN FELIPE,1.006760e+13,6.0,BUENOS AIRES,201801
1,2018-01-05,11013/2016,748.0,HOSPITAL LOCAL GENERAL DE AGUDOS DR. ARTURO MELO,1.006430e+13,6.0,BUENOS AIRES,201801
2,2018-01-05,11017/2016,1053.0,HOSPITAL LOCAL GENERAL DE AGUDOS DR. ARTURO MELO,1.006430e+13,6.0,BUENOS AIRES,201801
3,2018-01-05,21551/2017,2915.0,HOSPITAL INTERZONAL GENERAL DE AGUDOS DR. FIORITO,1.006040e+13,6.0,BUENOS AIRES,201801
4,2018-01-05,21818/2017,5777.0,HOSPITAL ZONAL ESPECIALIZADO EN ONCOLOGIA LUCI...,1.406600e+13,6.0,BUENOS AIRES,201801


Crear base de datos y estructuras

In [27]:
os.makedirs("database", exist_ok=True)
db_path = "database/hospitales.db"

# Conexión a SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Crear estructura de tabla
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS facturacion (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fecha_deposito DATE,
    numero_expediente TEXT,
    importe_aprobado REAL,
    nombre_hospital TEXT,
    codigo_refes NUMERIC,
    id_provincia INTEGER,
    provincia TEXT,
    periodo INTEGER
);
""")

conn.commit()

# Cargar datos
df.to_sql("facturacion", conn, if_exists="replace", index=False)

# Confirmar registros
count = cursor.execute(f"SELECT COUNT(*) FROM facturacion").fetchone()[0]
print(f"Cargados {count} registros en 'facturacion'.")

#conn.close()

Cargados 14935 registros en 'facturacion'.


Crear dashboard interactivo

In [5]:
# Importe total por provincia
query = """
SELECT provincia, SUM(importe_aprobado) AS total
FROM facturacion
GROUP BY provincia
ORDER BY total DESC
"""
df_prov = pd.read_sql(query, conn)

# Gráfico interactivo
fig1 = px.bar(df_prov, x="provincia", y="total")
fig1.update_layout(
    title="Importe total aprobado por provincia",
    xaxis_title="Provincia",
    yaxis_title="Importe total aprobado ($)"
)
fig1.show()

# Exportación a HTML
html_fig1 = fig1.to_html(full_html=False, include_plotlyjs='cdn')


In [6]:
query = """
SELECT provincia, nombre_hospital, SUM(importe_aprobado) AS total
FROM facturacion
GROUP BY provincia, nombre_hospital
ORDER BY total DESC
LIMIT 10
"""
df_hosp = pd.read_sql(query, conn)

fig2 = px.bar(
    df_hosp,
    x="nombre_hospital",
    y="total",
    color="provincia",
    hover_data=["provincia", "total"],
    labels={"provincia": "Provincia"}
)

fig2.update_layout(
    title="Importe total aprobado por hospital y provincia",
    xaxis_title="Hospital",
    yaxis_title="Importe total aprobado ($)",
    xaxis={'categoryorder':'total descending'}
)

fig2.show()

# Exportación a HTML
html_fig2 = fig2.to_html(full_html=False, include_plotlyjs=False)

In [16]:
# Evolución temporal
query = """
SELECT
    periodo,
    provincia,
    SUM(importe_aprobado) AS total
FROM facturacion
GROUP BY provincia, periodo
ORDER BY periodo
"""
df_monthly = pd.read_sql(query, conn)

# Crear el acumulado "TODAS"
df_total = df_monthly.groupby('periodo', as_index=False)['total'].sum()
df_total['provincia'] = 'TODAS'

# Concatenar con el DataFrame original
df_dropdown = pd.concat([df_monthly, df_total], ignore_index=True)

# Crear la figura inicial mostrando el acumulado "TODAS"
fig3 = px.line(
    df_dropdown[df_dropdown['provincia'] == 'TODAS'],
    x="periodo",
    y="total",
    title="Evolución mensual del importe aprobado - TODAS las provincias"
)

# Construir el dropdown
provincias = ['TODAS'] + list(df_monthly['provincia'].unique())
buttons = []

for prov in provincias:
    df_plot = df_dropdown[df_dropdown['provincia'] == prov]
    buttons.append(dict(
        label=prov,
        method='update',
        args=[{'x': [df_plot['periodo']],
               'y': [df_plot['total']],
               'mode': 'lines+markers'},
              {'title': f"Evolución mensual - {prov}"}]
    ))

fig3.update_layout(
    xaxis_title="Período",
    yaxis_title="Importe total aprobado ($)",
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        showactive=True,
        x=1.15,
        y=1.15
    )]
)

fig3.update_traces(mode="lines+markers")
fig3.update_xaxes(tickangle=30)

fig3.show()

# Exportación a HTML
html_fig3 = fig3.to_html(full_html=False, include_plotlyjs=False)

conn.close()


Exportación a HTML

In [24]:
# Crear carpeta html si no existe
os.makedirs("html", exist_ok=True)

# Crear HTML completo con estilo formal y diseño responsivo
html_dashboard = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dashboard de Facturación</title>
    <style>
        body {{
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f9f9f9;
            color: #333;
        }}
        header {{
            background-color: #f9f9f9;
            padding: 20px;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 20px;
            flex-wrap: wrap;
        }}
        header img {{
            height: 60px;
        }}
        header h1 {{
            margin: 0;
            font-size: 1.8em;
            text-align: center;
        }}
        section {{
            padding: 20px;
            margin: 10px auto;
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            max-width: 1200px;
        }}
        h1, h2 {{
            margin: 0 0 10px 0;
        }}
        .chart {{
            width: 100%;
            overflow-x: auto;
        }}
    </style>
</head>
<body>

<header>
    <img src="https://universidaddelaciudad.bue.edu.ar/pluginfile.php/1/theme_universidadelaciudad/logo/1756916448/Udelaciudad-logo-preferencial_de%20Bs%20As.png" alt="Logo Udelaciudad">
    <h1>Dashboard de Facturación de Hospitales y Provincias</h1>
    <p>Visualización interactiva de importes aprobados</p>
</header>

<section>
    <h2>Importe total por provincia</h2>
    <div class="chart">
        {html_fig1}
    </div>
</section>

<section>
    <h2>Ranking de 10 hospitales con mayor importe aprobado</h2>
    <div class="chart">
        {html_fig2}
    </div>
</section>

<section>
    <h2>Evolución mensual del importe aprobado</h2>
    <div class="chart">
        {html_fig3}
    </div>
</section>

</body>
</html>
"""

# Guardar el dashboard en un solo archivo HTML
with open("html/dashboard_facturacion.html", "w", encoding="utf-8") as f:
    f.write(html_dashboard)
    f.close()

# Llamar al archivo html exportado (Dashboard)
HTML("html/dashboard_facturacion.html")
